# Imports

In [1]:
from functools import reduce
from pathlib import Path
import time
import datetime

import pandas as pd
import requests
from wmfdata import hive
from wmfdata.utils import print_err, pd_display_all

You are using wmfdata v1.0.3, but v1.0.4 is available.

To update, run `pip install --upgrade git+https://github.com/neilpquinn/wmfdata.git@release`.

To see the changes, refer to https://github.com/neilpquinn/wmfdata/blob/release/CHANGELOG.md


# Parameters

In [2]:
# TSV file where metrics are or will be saved
FILENAME = "metrics/diversity_metrics.tsv"

# Metric month. The mediawiki_history snapshot must be from the metrics month or later.
# METRICS_MONTH_TEXT = '2019-07'
# MEDIAWIKI_HISTORY_SNAPSHOT = '2019-07'

last_month = datetime.date.today().replace(day=1) - datetime.timedelta(days=1)

METRICS_MONTH_TEXT = last_month.strftime("%Y-%m")
MEDIAWIKI_HISTORY_SNAPSHOT = last_month.strftime("%Y-%m")


# Preparation

In [3]:
# Convert our metrics month to all the formats we need and provide them in a dict
# so we can easily use them to format strings
metrics_month = pd.Period(METRICS_MONTH_TEXT)
date_params = {
    "mediawiki_history_snapshot": MEDIAWIKI_HISTORY_SNAPSHOT,
    "metrics_month": str(metrics_month), 
    "metrics_month_first_day": str(metrics_month.asfreq("D", how="start")),
    "metrics_next_month_first_day": str((metrics_month+1).asfreq("D", how="start")),
    "metrics_month_end": str((metrics_month + 1).start_time),
    "metrics_month_last_day": str(metrics_month.asfreq("D", how="end")),
    "api_metrics_month_first_day": metrics_month.asfreq("D", how="start").strftime("%Y%m%d"),
    "api_metrics_month_day_after": (metrics_month + 1).asfreq("D", how="start").strftime("%Y%m%d"),
    "metrics_prev_month": str(metrics_month - 1),
    "retention_cohort": str(metrics_month - 2)
}

# Load any previous results
try:
    old_metrics = (
        pd.read_csv(FILENAME, sep="\t", parse_dates = ["month"])
        .set_index("month")
    )
except FileNotFoundError:
    old_metrics = None
    
def prepare_query(filename):
    return (
        Path(filename)
        .read_text()
        .format(**date_params)
    )

# MariaDB and Hive query metrics

In [4]:
queries = {
    "global_south_edits_editors": {
        "file": "queries/global_south_edits_editors.hql",
        "engine": "hive"
    },

    "global_south_new_editor_retention": {
        "file": "queries/global_south_new_editor_retention.hql",
        "engine": "hive"
    },
    
    "global_south_net_new_content": {
        "file": "queries/global_south_net_new_content.hql",
        "engine": "hive"
    },
    
    "global_south_wikidata_entities": {
        "file": "queries/global_south_net_new_wikidata.hql",
        "engine": "hive"
    },
    
    "global_north_edits_editors": {
        "file": "queries/global_north_edits_editors.hql",
        "engine": "hive"
    },

    "global_north_new_editor_retention": {
        "file": "queries/global_north_new_editor_retention.hql",
        "engine": "hive"
    },
    
    "global_north_net_new_content": {
        "file": "queries/global_north_net_new_content.hql",
        "engine": "hive"
    },
    "global_north_wikidata_entities": {
        "file": "queries/global_north_net_new_wikidata.hql",
        "engine": "hive"
    }
}


for key, val in queries.items():
    query = prepare_query(val["file"])
    engine = val["engine"]
    print_err("Running {} on {}...".format(key, engine))
    
    if engine == "mariadb":
        result = mariadb.run(query)
    elif engine == "hive":
        result = hive.run(query)
    else:
        raise ValueError("Unknown engine specified.") 
    
    result = result.assign(month=lambda df: pd.to_datetime(df["month"]))
    val["result"] = result

Running global_south_edits_editors on hive...
Running global_south_new_editor_retention on hive...
Running global_south_net_new_content on hive...
Running global_south_wikidata_entities on hive...
Running global_north_edits_editors on hive...
Running global_north_new_editor_retention on hive...
Running global_north_net_new_content on hive...
Running global_north_wikidata_entities on hive...


# Combining and saving metrics

In [5]:
# Assemble list of result dataframes
results = [val["result"] for _, val in queries.items()]


In [6]:
# Merge them all, assuming that the month is the only common column
new_metrics = reduce(lambda l, r: pd.merge(l, r, how="outer"), results)

# Set the month as an index so combine_first works properly
new_metrics = new_metrics.set_index("month").sort_index()
new_metrics = new_metrics.iloc[:,:].astype(float)

if old_metrics is None:
    metrics = new_metrics
else:
    metrics = new_metrics.combine_first(old_metrics)
    
pd_display_all(metrics.tail())

,global_north_active_editors,global_north_edits,global_north_net_new_content,global_north_new_editor_retention,global_north_nonbot_edits,global_north_wikidata_entities,global_south_active_editors,global_south_edits,global_south_net_new_content,global_south_new_editor_retention,global_south_nonbot_edits,global_south_wikidata_entities
month,,,,,,,,,,,,
2020-03-01,73738.0,22969292.0,2508475.0,0.086572,12125525.0,1526020.0,21524.0,2802052.0,410424.0,0.042328,2441182.0,92564.0
2020-04-01,78261.0,25342753.0,2136987.0,0.075760,14170137.0,1275354.0,23011.0,3374326.0,412606.0,0.041195,2762677.0,40889.0
2020-05-01,81053.0,27306458.0,2666812.0,0.080915,14020678.0,1371480.0,25316.0,4189735.0,367585.0,0.049743,2998223.0,48740.0
2020-06-01,74954.0,25704657.0,1371225.0,0.077205,11772149.0,167523.0,23312.0,3417381.0,421614.0,0.051572,2516055.0,24707.0
2020-07-01,71585.0,24853975.0,1431825.0,0.064619,11419192.0,332579.0,23467.0,3133019.0,395942.0,0.042145,2721998.0,21583.0


In [7]:
metrics.to_csv(FILENAME, sep="\t")